## 1. This patient may have sepsis
<p>Sepsis is a deadly syndrome where a patient has a severe infection that causes organ failure. The sooner septic patients are treated, the more likely they are to survive, but sepsis can be challenging to recognize. It may be possible to use hospital data to develop machine learning models that could flag patients who are likely to be septic. However, before we develop predictive algorithms, we need a reliable method to determine patients who are septic. One component of sepsis is a severe infection.</p>
<p>In this project, we will use two weeks of hospital electronic health record (EHR) data to find out which patients had a severe infection according to four criteria. We will look into the data to see if a doctor ordered a blood test to look for bacteria (a blood culture) and gave the patient a series of intervenous antibiotics. </p>
<p>Let's get started!</p>

In [0]:
# Load packages
# .... YOUR CODE FOR TASK 1 ....

# Read in the data
antibioticDT <- ....

# Look at the first 30 rows
# .... YOUR CODE FOR TASK 1 ....

## 2. Which antibiotics are "new"?
<p>These data represent all drugs administered in a hospital over two weeks. Each row represents one time a patient was given an antibiotic. The variables include the patient identification number, the day the drug was administered, the name of the antibiotic, and how it was administered. For example, patient "8" received doxycycline by mouth on the first day of their stay.</p>
<p>We will identify patients with a serious infection using the following criteria. </p>
<p><strong>Criteria for Suspected Infection</strong><a href="https://www.ncbi.nlm.nih.gov/pubmed/28903154">*</a></p>
<ol>
<li>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</li>
<li>The sequence must start with a new antibiotic, defined as an antibiotic type that was not given in the previous two days.</li>
<li>The sequence must start within two days of a blood culture.</li>
<li>There must be at least one intervenous (I.V.) antibiotic within the +/-2 day window.</li>
</ol>
<p>Let's start with the second item by finding which rows represent "new antibiotics". We will determine if each antibiotic was given to the patient in the prior two days. We'll visualize this task by looking at the data sorted by id, antibiotic type, and day.</p>

In [0]:
# Sort the data and examine the first 40 rows
# .... YOUR CODE FOR TASK 2 .....

# Use shift to calculate the last day a particular drug was administered
antibioticDT[ , last_administration_day := ...., 
  by = .(patient_id, antibiotic_type)]

# Calculate the number of days since the drug was last administered
antibioticDT[ , days_since_last_admin := ....]

# Create antibiotic_new with an initial value of one, then reset it to zero as needed
# .... YOUR CODE FOR TASK 2 .....

## 3. Looking at the blood culture data
<p>Now let's look at blood culture data from the same two-week period in this hospital. These data are in <code>blood_cultureDT.csv</code>. Let's start by reading it into the workspace and having a look at a few rows. </p>
<p>Each row represents one blood culture and gives the patient's id and the day the blood culture test occurred. For example, patient "8" had a blood culture on the second day of their hospitalization and again on the thirteenth day. Notice that some patients from the antibiotic dataset are not in this dataset and vice versa. Some patients are in neither because they received neither antibiotics nor a blood culture.</p>

In [0]:
# Read in blood_cultureDT.csv
blood_cultureDT <- ....

# Print the first 30 rows
# .... YOUR CODE FOR TASK 3 ....

## 4. Combine the antibiotic data and the blood culture data
<p>To find which antibiotics were given close to a blood culture test, we need to combine the drug administration data with the blood culture data. We'll keep only patients that are still candidates for infection&mdash;only those in both data sets.</p>
<p>A challenge with the data is that some patients had blood cultures on several different days. For each of those days, we will see if there is a sequence of antibiotic days close to them. To accomplish this, in the merge we will match each blood culture to each antibiotic day.</p>
<p>After sorting the data following the merge, you will see that each patient's antibiotic sequence repeats for each blood culture day. This repetition allows us to look at each blood culture day and check if it is associated with a qualifying sequence of antibiotics.</p>

In [0]:
# Merge antibioticDT with blood_cultureDT
combinedDT <- ....

# Sort by patient_id, blood_culture_day, day_given, and antibiotic_type
# .... YOUR CODE FOR TASK 4 .....

# Print and examine the first 30 rows
# .... YOUR CODE FOR TASK 4 .....

## 5. Determine whether each row is in-window
<p>Now that we have the antibiotic and blood culture data combined, we can test each drug administration against each blood culture to see if it's "in the window."</p>

In [0]:
# Make a new variable called drug_in_bcx_window
# ....YOUR CODE FOR TASK 5 ....

## 6. Check the I.V. requirement
<p>Now let's look at the fourth item in the criteria. </p>
<p><strong>Criteria for Suspected Infection</strong><a href="https://www.ncbi.nlm.nih.gov/pubmed/28903154">*</a></p>
<ol>
<li>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</li>
<li>The sequence must start with a new antibiotic, defined as an antibiotic type that was not given in the previous two days.</li>
<li>The sequence must start within two days of a blood culture.</li>
<li><em>There must be at least one intervenous (I.V.) antibiotic within the +/-2 day window.</em></li>
</ol>

In [0]:
# Create a variable indicating if there was at least one I.V. drug given in the window
# .... YOUR CODE FOR TASK 6 ....

# Exclude rows in which the blood_culture_day does not have any I.V. drugs in window 
# .... YOUR CODE FOR TASK 6 ....

## 7. Find the first day of possible sequences
<p>We're getting close! Let's review the criteria again.</p>
<p><strong>Criteria for Suspected Infection</strong><a href="https://www.ncbi.nlm.nih.gov/pubmed/28903154">*</a></p>
<ol>
<li>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</li>
<li>The sequence must start with a new antibiotic, defined as an antibiotic type that was not given in the previous two days.</li>
<li>The sequence must start within two days of a blood culture.</li>
<li>There must be at least one intervenous (I.V.) antibiotic within the +/-2 day window.</li>
</ol>
<p>Let's assess the first criterion by finding the first day of possible 4-day qualifying sequences.    </p>

In [0]:
# Create a new variable called day_of_first_new_abx_in_window
# .... YOUR CODE FOR TASK 7 ....

# Remove rows where the day is before this first qualifying day
# .... YOUR CODE FOR TASK 7 ....

## 8. Simplify the data
<p>The first criterion is: <em>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</em></p>
<p>We've pinned down the first day of possible sequences in the previous task. Now we have to check for four-day sequences. We don't need the drug type (name); we need the days the antibiotics were administered.</p>

In [0]:
# Create a new data.table containing only patient_id, blood_culture_day, and day_given
simplified_data <- ....

# Remove duplicate rows
#.... YOUR CODE FOR TASK 8 ....

## 9. Extract first four rows for each blood culture
<p>To check for four-day sequences, let's pull out the first four days (rows) for each patient/blood culture combination. Some patients will have less than four antibiotic days. We'll remove them first.</p>

In [0]:
# Count the antibiotic days within each patient/blood culture day combination
# .... YOUR CODE FOR TASK 9 ....

# Remove blood culture days with less than four rows 
simplified_data <- ....

# Select the first four days for each blood culture
first_four_days <- .... 

## 10. Consecutive sequence
<p>Now we need to check whether each four-day sequence qualifies by having no gaps of more than one day.</p>
<!--"Patient receives antibiotics for a sequence of 4 days, with gaps of 1 day allowed."-->

In [0]:
# Make the indicator for consecutive sequence
# .... YOUR CODE FOR TASK 9 ....

## 11. Select the patients who meet criteria
<p>A patient would meet the criteria if any of their blood cultures were accompanied by a qualifying sequence of antibiotics. Now that we've determined which each blood culture qualify let's select the patients who meet the criteria.</p>

In [0]:
# Select the rows which have four_in_seq equal to 1
suspected_infection <- first_four_days[.... == ....]

# Retain only the patient_id column
# .... YOUR CODE FOR TASK 9 ....

# Remove duplicates
# .... YOUR CODE FOR TASK 9 ....

# Make an infection indicator
# .... YOUR CODE FOR TASK 9 ....

## 12. Find the prevalence of sepsis
<p>In this project, we used two EHR datasets to flag patients who were suspected of having a serious infection. We also got a <code>data.table</code> workout!</p>
<p>So far, we've been looking at records of all antibiotics administered and blood cultures that occurred over two weeks at a particular hospital. However, not all patients who were hospitalized over this period are represented in <code>combinedDT</code> because not all of them took antibiotics or had blood culture tests. We have to read in and merge the rest of the patient information to see what percentage of patients at the hospital might have had a serious infection.</p>

In [0]:
# Read in "all_patients.csv"
all_patientsDT <- ....

# Merge this with the infection flag data
all_patientsDT <- ....

# Set any missing values of the infection flag to 0
# .... YOUR CODE FOR TASK 12 ....

# Calculate the percentage of patients who met the criteria for presumed infection
ans  <- .... 
ans